# Initialization

In [ ]:
import bgk

In [ ]:
path = f"/mnt/lustre/IAM851/jm1667/psc-runs/case1/trials/exact/B00.25-n512/"

B = bgk.readParam(path, "H_x", float)
res = bgk.readParam(path, "n_grid", int)
ve_coef = bgk.readParam(path, "v_e_coef", float)
input_path = bgk.readParam(path, "path_to_data", str)

struct_radius = bgk.Input(input_path).get_radius_of_structure()

wholeSlice = bgk.DataSlice(slice(None, None), "")
centerSlice = bgk.DataSlice(slice(-struct_radius, struct_radius), "Central ")

loader = bgk.Loader(path, engine="pscadios2", species_names=['e', 'i'])
size = loader._get_xr_dataset("pfd", 0).length[1]

print(f"B={B}")
print(f"res={res}")
print(f"size={size}")
print(f"struct size={2*struct_radius:.3f}")
print(f"ve_coef={ve_coef}")
print(f"input_path={input_path}")

In [ ]:
# fiddle with this until as many steps as possible are used (usually, they can all be used)
nframes = 100

videoMaker = bgk.VideoMaker(nframes, loader)

completion_percent = 100 * loader.fields_max / loader.nmax
video_coverage_percent = 100 * nframes * videoMaker.fields_stepsPerFrame / loader.fields_max
steps_used_percent = 100 * nframes / (loader.fields_max / loader.fields_every)
print(f"steps simulated:      {loader.fields_max} ({completion_percent:.1f}% complete)")
print(f"nframes in animation: {nframes}")
print(f"steps per frame:      {videoMaker.fields_stepsPerFrame}")
print(f"max step in video:    {nframes * videoMaker.fields_stepsPerFrame} ({video_coverage_percent:.1f}% coverage, {steps_used_percent:.1f}% step used)")
if video_coverage_percent != 100:
    print(f"suggested nframes:    {loader.get_all_suggested_nframes(nframes)[0]}")

# Animation
from https://stackoverflow.com/questions/18743673/show-consecutive-images-arrays-with-imshow-as-repeating-animation-in-python
and http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-notebooks/

In [ ]:
from IPython.display import HTML
import bgk.run_params as rp

In [ ]:
# select parameter
param = rp.ne
print(f"quantity: {param.title}")

In [ ]:
# load data
videoMaker.loadData(param)

In [ ]:
# slice data
sliceId = 0
whichSlice = [wholeSlice, centerSlice][sliceId]
videoMaker.setSlice(whichSlice)

print(f"view: {whichSlice.viewAdjective}= {whichSlice.slice}")

In [ ]:
# view t=0
%matplotlib widget
fig, ax, im = videoMaker.viewFrame(77)
fig

In [ ]:
# make movie
%matplotlib widget
anim = videoMaker.viewMovie(fig, ax, im)
HTML(anim.to_html5_video())

# Spiral

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np

In [ ]:
maxR = size / 2
maxT = videoMaker.times[-1]
deltaR = .003

thetaGrid = videoMaker.xGrid

def getRslice(data: xr.DataArray, r: float) -> xr.DataArray:
    return data.where((r <= videoMaker.rGrid) & (videoMaker.rGrid < r + deltaR))

def flattenRslice(rslice: xr.DataArray) -> np.ndarray:
    rslice = rslice.data.flatten()
    return rslice[~np.isnan(rslice)]

In [ ]:
r = .005

## Choose Time

In [ ]:
t = 50

idx = int(t / maxT * nframes)

ne = getRslice(videoMaker.slicedDatas[idx], r)
theta = getRslice(np.arctan2(ne.z, ne.y), r)
print(idx)

In [ ]:
%matplotlib inline
plt.close("all")
im = plt.imshow(ne)
# im = plt.imshow(theta)
plt.colorbar(im)

In [ ]:
theta_axis = flattenRslice(theta)
ne_axis = flattenRslice(ne)

### Scatter at time

In [ ]:
plt.close("all")
plt.scatter(theta_axis, ne_axis, s=10)

### Binning at time

In [ ]:
theta_bins = np.linspace(theta_axis.min(), theta_axis.max(), 100)
bin_idxs = np.digitize(theta_axis, theta_bins)
ne_means = [ne_axis[bin_idxs == i].mean() for i in range(1, len(theta_bins)+1)]

In [ ]:
plt.close("all")
plt.plot(theta_bins, ne_means)

### FFT at time

In [ ]:
import scipy.signal as sig

In [ ]:
idx_freq, power = sig.periodogram(ne_means, nfft=len(ne_means) * 4)
freq = idx_freq * len(theta_bins) / 6.28

In [ ]:
plt.close("all")
plt.xlabel("Frequency")
plt.ylabel("Amplitude")
plt.plot(freq, power)

In [ ]:
search_cutoff = len(power) // 2
peak_freq = freq[power[search_cutoff:].argmax() + search_cutoff]
print(f"peak frequency: {peak_freq}")

## Arbitrary Shift Finder

In [ ]:
thetas = np.linspace(-np.pi, np.pi, 100)
rslice = getRslice(videoMaker.slicedDatas[0], r)
bin_idxs = np.digitize(flattenRslice(getRslice(np.arctan2(rslice.z, rslice.y), r)), thetas)

In [ ]:
def interpolate_in_nes(thetas: np.ndarray, nes: np.ndarray, theta: float) -> float:
    idx = (thetas > theta).argmax()
    dtheta1 = thetas[idx] - theta
    dtheta2 = theta - thetas[idx-1]
    return (nes[idx] * dtheta2 + nes[idx-1] * dtheta1) / (dtheta1 + dtheta2)

def find_shift(thetas: np.ndarray, nes1: np.ndarray, nes2: np.ndarray, n_iterations=3, shift_search=None):
    # doesn't work for some reason:
    # def f(theta: float, shift: float) -> float:
    #     return interpolate_in_nes(thetas, nes1, theta + shift)

    # [popt, pcov] = opt.curve_fit(f, thetas, nes2, p0=[0], bounds=([-np.pi], [np.pi]))
    # [shift] = popt


    best_shift = np.nan
    best_sum_square_errors = np.inf
    shift_search = shift_search if shift_search is not None else np.linspace(-np.pi/2, np.pi/2, len(thetas))
    for shift in shift_search:
        this_sum_square_errors = 0
        for theta, ne2 in zip(thetas, nes2):
            this_sum_square_errors += (interpolate_in_nes(thetas, nes1, theta + shift) - ne2) ** 2
        if this_sum_square_errors < best_sum_square_errors:
            best_shift, best_sum_square_errors = shift, this_sum_square_errors

    if n_iterations <= 1:
        return best_shift

    dtheta = shift_search[1] - shift_search[0]
    return find_shift(thetas, nes1, nes2, n_iterations-1, np.linspace(best_shift - 4*dtheta, best_shift + 4*dtheta, 10))

def get_nes(bin_idxs: np.ndarray, frame: int) -> np.ndarray:
    ne_axis = flattenRslice(getRslice(videoMaker.slicedDatas[frame], r))
    return [ne_axis[bin_idxs == i].mean() for i in range(1, max(bin_idxs) + 1)]

def t2i(t: float) -> int:
    return int(t * nframes / max(videoMaker.times))

In [ ]:
tstart = 0
tstop = None
istart, istop = t2i(tstart), (t2i(tstop) if tstop else nframes)

In [ ]:
# Find shifts
nes2 = get_nes(bin_idxs, istart)
shifts = []
for frame in range(istart+1, istop):
    nes1, nes2 = nes2, get_nes(bin_idxs, frame)
    shifts.append(find_shift(thetas, nes1, nes2, n_iterations=5))

shifts = np.array(shifts)

phase_velocities = shifts / videoMaker.times[1]

phase_angles = [0]
for shift in shifts:
    phase_angles.append(phase_angles[-1] + shift)

In [ ]:
# plot phase velocity vs time
plt.close("all")
plt.plot(videoMaker.times[istart+1:istop], phase_velocities)
plt.xlabel("time")
plt.ylabel("phase velocity")
plt.show()

In [ ]:
# plot phase angle vs time
plt.close("all")
plt.plot(videoMaker.times[istart:istop], phase_angles)
plt.xlabel("time")
plt.ylabel("phase angle")
plt.show()

In [ ]:
# phase velocity in select region
phase_velocities[t2i(30):].mean()